In [1]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
# Ensure compatibility across different OpenCV versions
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Load COCO class labels
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]


# Initialize video
cap = cv2.VideoCapture('people_walking.mp4')
frame_id = 0
unique_people_counted = 0

while True:
    ret, frame = cap.read()
    frame_id += 1

    if not ret:
        break

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # 0 is the ID for 'person' in COCO
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    people_count = len(indexes)
    
    # For simplicity, this example just counts unique detections as new people
    # A real application would use tracking to count people more accurately
    unique_people_counted += people_count
    
    print(f"Frame {frame_id}: {people_count} people detected. Total so far: {unique_people_counted}")

    # Visualization of the results of a detection.
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Image", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Frame 1: 38 people detected. Total so far: 38
Frame 2: 37 people detected. Total so far: 75
Frame 3: 39 people detected. Total so far: 114
Frame 4: 38 people detected. Total so far: 152
Frame 5: 39 people detected. Total so far: 191
